In [11]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [12]:
urls = ['https://www.basketball-reference.com/players/s/sabondo01.html',
       'https://www.basketball-reference.com/players/r/redicjj01.html',
       'https://www.basketball-reference.com/players/b/bogdabo02.html',
       'https://www.basketball-reference.com/players/b/bogdabo01.html',
       'https://www.basketball-reference.com/players/d/doncilu01.html',
       'https://www.basketball-reference.com/players/j/jokicni01.html',
       'https://www.basketball-reference.com/players/p/porzikr01.html',
       'https://www.basketball-reference.com/players/v/vucevni01.html',
       'https://www.basketball-reference.com/players/n/nurkiju01.html',
       'https://www.basketball-reference.com/players/s/saricda01.html',
       'https://www.basketball-reference.com/players/w/westbru01.html',
       'https://www.basketball-reference.com/players/d/derozde01.html',
       'https://www.basketball-reference.com/players/g/griffbl01.html',
       'https://www.basketball-reference.com/players/l/lavinza01.html',
       'https://www.basketball-reference.com/players/g/gordoaa01.html',
       'https://www.basketball-reference.com/players/g/gasolma01.html',
       'https://www.basketball-reference.com/players/h/holidjr01.html',
       'https://www.basketball-reference.com/players/j/jonesde02.html',
       'https://www.basketball-reference.com/players/p/paulch01.html',
       'https://www.basketball-reference.com/players/w/walljo01.html',
       'https://www.basketball-reference.com/players/i/inglejo01.html',
       'https://www.basketball-reference.com/players/g/gallida01.html',
       'https://www.basketball-reference.com/players/v/valanjo01.html',
       'https://www.basketball-reference.com/players/g/gasolpa01.html',
       'https://www.basketball-reference.com/players/p/parketo01.html',
       'https://www.basketball-reference.com/players/n/nowitdi01.html',
       'https://www.basketball-reference.com/players/d/duncati01.html',
       'https://www.basketball-reference.com/players/b/bryanko01.html',
       'https://www.basketball-reference.com/players/s/stojape01.html']
player = ['Domantas Sabonis','JJ Redick','Bojan Bogdanovic','Bodgan Bogdanovic',
          'Luka Doncic','Nikola Jokic','Kristaps Porzingis','Nikola Vucevic',
          'Jusuf Nurkic','Dario Saric','Russell Westbrook','Demar Derozan','Blake Griffen',
         'Zach Lavine','Aaron Gordon','Marc Gasol','Jrue Holiday','Derick Jones Jr.',
          'Chris Paul','John Wall','Joe Ingles','Danilo Gallinari','Jonas Valanciunas','Pau Gasol',
          'Tony Parker','Dirk Nowitzky','Tim Duncan','Kobe Bryant','Peja Stojakovic']

stats = ['g','fga_per_g','fg3_pct',
         'fg2_pct','efg_pct','ft_pct','tov_per_g','orb_per_g']
stat = ['fga_per_g','fg3_pct',
         'fg2_pct','efg_pct','ft_pct','tov_per_g','orb_per_g']


In [13]:
games = []
fga_per_g = []
fg3_pct = []
fg2_pct = []
efg_pct = []
ft_pct = []
tov_per_g = []
orb_per_g = []

hdf = pd.DataFrame()
for i in range(len(urls)):
    b_page = requests.get(urls[i])
    b_soup = BeautifulSoup(b_page.content, 'html.parser')
    stats_list = [[td.getText() for td in b_soup.findAll('td', {'data-stat': stat})] for stat in stats]
    df = pd.DataFrame(stats_list).T
    df.columns = stats
    #gets rid of row with empty spaces
    df = df[df.g != '']
    #convert games column from string to int
    df['g'] = df['g'].astype(int)
    #find max games row which is the row with averages per player
    df = df.iloc[df['g'].idxmax()]
    games.append(df[0])
    fga_per_g.append(df[1])
    fg3_pct.append(df[2])
    fg2_pct.append(df[3])
    efg_pct.append(df[4])
    ft_pct.append(df[5])
    tov_per_g.append(df[6])
    orb_per_g.append(df[7])

In [14]:
hdf['player'] = player
hdf['games'] = games
hdf['fga_per_g'] = fga_per_g
hdf['fg3_pct'] = fg3_pct
hdf['fg2_pct'] = fg2_pct
hdf['efg_pct'] = efg_pct
hdf['ft_pct'] = ft_pct
hdf['tov_per_g'] = tov_per_g
hdf['orb_per_g'] = orb_per_g
hdf
colum = list(hdf.columns)
hdf = hdf[['player','games',
 'fga_per_g',
 'fg3_pct',
 'fg2_pct',
 'efg_pct',
 'ft_pct',
 'tov_per_g',
 'orb_per_g']]


In [15]:
hdf = hdf.set_index('player')
hdf = hdf.drop(['games'], axis=1)

In [16]:
#standardize data due to varying units across features
scaler = StandardScaler()
scaled_df = scaler.fit_transform(hdf)

sdf = pd.DataFrame(data = scaled_df, index = player, columns = stat)
sdf.to_csv('sdf.csv')